# Import Dependencies

In [4]:
import numpy as np
import pandas as pd
import plotly
import mysql.connector
from dotenv import load_dotenv
import os

load_dotenv()

True

In [17]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=os.getenv("MYSQL_DB_PASSWORD"),
    database="imdb_star"
)

cursor = conn.cursor()
cursor.execute("SELECT * FROM dimepisode LIMIT 30")
row = cursor.fetchall()
print(row)

conn.close()

[('tt0031458', 'tt32857063', None, None), ('tt0041951', 'tt0041038', 1, 9), ('tt0042816', 'tt0989125', 1, 17), ('tt0042889', 'tt0989125', None, None), ('tt0043426', 'tt0040051', 3, 42), ('tt0043631', 'tt0989125', 2, 16), ('tt0043693', 'tt0989125', 2, 8), ('tt0043710', 'tt0989125', 3, 3), ('tt0044093', 'tt0959862', 1, 6), ('tt0044668', 'tt0044243', 2, 16), ('tt0044901', 'tt0989125', 3, 46), ('tt0045519', 'tt0989125', 4, 11), ('tt0045960', 'tt0044284', 2, 3), ('tt0046135', 'tt0989125', 4, 5), ('tt0046150', 'tt0341798', None, None), ('tt0046855', 'tt0046643', 1, 4), ('tt0046864', 'tt0989125', 5, 20), ('tt0047810', 'tt0914702', 3, 36), ('tt0047852', 'tt0047745', 1, 15), ('tt0047858', 'tt0046637', 2, 9), ('tt0047961', 'tt0989125', 6, 5), ('tt0048067', 'tt0046587', 2, 20), ('tt0048302', 'tt0047768', 1, 6), ('tt0048371', 'tt0989125', 6, 11), ('tt0048378', 'tt0047702', 1, 6), ('tt0048462', 'tt0047702', 1, 3), ('tt0048562', 'tt0047768', 1, 10), ('tt0048883', 'tt0989125', 6, 42), ('tt0049025', '